In [1]:
import numpy as np
import torch.nn as nn
import torch
import numpy as np
from transformers import AutoTokenizer
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
file_name = "nudity_train1000.pkl"
data = pd.read_pickle(f'training_pkl/{file_name}')
print(data.head)
print(data.columns)

<bound method NDFrame.head of        IMDb_id                                                url None Mild  \
0    tt0111438  https://www.imdb.com/title/tt0111438/parentalg...    2   14   
1    tt0349080  https://www.imdb.com/title/tt0349080/parentalg...    5    4   
2    tt0088680  https://www.imdb.com/title/tt0088680/parentalg...   11   47   
3    tt0066065  https://www.imdb.com/title/tt0066065/parentalg...    3    4   
4    tt0065481  https://www.imdb.com/title/tt0065481/parentalg...    0    5   
..         ...                                                ...  ...  ...   
995  tt0112857  https://www.imdb.com/title/tt0112857/parentalg...    5   14   
996  tt0091886  https://www.imdb.com/title/tt0091886/parentalg...    1    7   
997  tt1228987  https://www.imdb.com/title/tt1228987/parentalg...   11   38   
998  tt0239986  https://www.imdb.com/title/tt0239986/parentalg...    7    3   
999  tt0068732  https://www.imdb.com/title/tt0068732/parentalg...    0   14   

    Moderate Severe  

In [3]:
data.head()

,IMDb_id,url,None,Mild,Moderate,Severe
0,tt0111438,https://www.imdb.com/title/tt0111438/parentalg...,2,14,20,17
1,tt0349080,https://www.imdb.com/title/tt0349080/parentalg...,5,4,6,59
2,tt0088680,https://www.imdb.com/title/tt0088680/parentalg...,11,47,50,15
3,tt0066065,https://www.imdb.com/title/tt0066065/parentalg...,3,4,0,0
4,tt0065481,https://www.imdb.com/title/tt0065481/parentalg...,0,5,6,0


In [5]:
# Pobranie kolumny 'IMDb_id' jako zmienna X
X = data['IMDb_id']
print(X)
# Pobranie kolumn 'None', 'Mild', 'Moderate', 'Severe' jako zmienna y
y = data[['None', 'Mild', 'Moderate', 'Severe']]
y =  y.astype(float)

0      tt0111438
1      tt0349080
2      tt0088680
3      tt0066065
4      tt0065481
         ...    
995    tt0112857
996    tt0091886
997    tt1228987
998    tt0239986
999    tt0068732
Name: IMDb_id, Length: 1000, dtype: object


In [6]:
X

0      tt0111438
1      tt0349080
2      tt0088680
3      tt0066065
4      tt0065481
         ...    
995    tt0112857
996    tt0091886
997    tt1228987
998    tt0239986
999    tt0068732
Name: IMDb_id, Length: 1000, dtype: object

In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train.dtypes

None        float64
Mild        float64
Moderate    float64
Severe      float64
dtype: object

In [139]:
y_train = [torch.tensor(y).float().unsqueeze(0) for y in y_train.values]
y_test = [torch.tensor(y).float().unsqueeze(0) for y in y_test.values]
#now get first 7 elements of each tensor in X_train and X_test

In [66]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\dimas\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dimas\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [90]:
text = "Here is the sentence I want embeddings for."
tokenized_text = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt')
X_train = [tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt') for text in X_train]
X_test = [tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt') for text in X_test]
input_ids_train = [x['input_ids'] for x in X_train]
input_ids_test = [x['input_ids'] for x in X_test]

In [91]:
attention_mask_train = [x['attention_mask'] for x in X_train]
attention_mask_test = [x['attention_mask'] for x in X_test]

In [141]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.bert = BertModel.from_pretrained('prajjwal1/bert-tiny')
        self.linear = nn.Linear(input_dim, 4)
    def forward(self, i):
        outputs = self.bert(input_ids=input_ids_train[i], attention_mask=attention_mask_train[i]).last_hidden_state 
        # Take the mean of the sequence_length dimension
        out = self.linear(outputs)
        out = out.mean(dim=1)
        return out

model = LinearRegression(128, 4)
print('Model')
# Loss function
criterion = nn.MSELoss()
print('Criterion')
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.0000001)
print('Optimizer')
# Forward pass
o = model.forward(1)
print('Forward pass')
criteria = criterion(o, y_train[0])

Model
Criterion
Optimizer
Forward pass


In [142]:
#train the model
num_epochs = 100
for epoch in range(num_epochs):
    for i in range(len(X_train)):
        output = model.forward(X_train[i])
        loss = criterion(output, y_train[i])
        
        loss.backward()
        
        optimizer.step()
        
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

TypeError: list indices must be integers or slices, not BatchEncoding